## Optimization model 

In [1]:
from model import shortestPathModel

In [2]:
# model for shortest path
grid = (5,5)
sp_model = shortestPathModel(grid)

Academic license - for non-commercial use only - expires 2021-04-13
Using license file C:\Users\Apocrypse\gurobi.lic


In [3]:
sp_model.setObj([1 for i in range(40)])
sp_model.solve()

([1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0],
 8.0)

## Data Loader

In [4]:
from data import shortestpath, dataset
from torch.utils.data import DataLoader

In [5]:
# generate data (features and costs)
n = 100
p = 5
x, c = shortestpath.genData(n, p, grid)

In [6]:
# get data set
sp_dataset = dataset.optDataset(sp_model, x, c)

Optimizing...


100%|███████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 283.66it/s]


In [7]:
# get data loader
sp_loader = DataLoader(sp_dataset, batch_size=32, shuffle=True)

In [8]:
for i, data in enumerate(sp_loader):
    x, c, w, z = data
    break

In [9]:
x.shape

torch.Size([32, 5])

In [10]:
c.shape

torch.Size([32, 40])

In [11]:
w.shape

torch.Size([32, 40])

In [12]:
z.shape

torch.Size([32, 1])

## Loss Function

In [15]:
from loss import SPOPlusLoss

In [ ]:
spoplus_loss = SPOPlusLoss()